| Version | Date       | Developer | Remark             |
|---------|------------|-----------|--------------------|
| 1.0     | Feb-1-2025 | Johnson | Initial version:developed pipeline & function for data engineering    |

In [0]:
# pip install synapseml
!pip install shap
!pip install pycaret

  Using cached shap-0.47.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (980 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached slicer-0.0.8-py3-none-any.whl (15 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl (20 kB)
  Using cached numba-0.60.0-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.7 MB)
  Using cached numpy-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.5 MB)
  Using cached llvmlite-0.43.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.9 MB)
  Using cached numpy-1.22.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Not uninstalling numpy at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-103b205b-fcdb-46b5-bb1c-c9c4b7eb8152
    Can't uninstall 'numpy'. No files were found to uninstall.
You should consider upgrading via the '/local

In [0]:
import numpy as np; np.__version__ = '1.24.0'
import shap
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline,PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *
from pyspark.sql.types import  *


from sklearn.metrics import *
from sklearn.model_selection  import train_test_split,GridSearchCV,cross_val_score
from sklearn.model_selection  import train_test_split
from sklearn.ensemble import RandomForestClassifier






/databricks/python/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, MinMaxScaler, VectorAssembler


categorical_columns = ['policy_state',
'policy_csl',
'insured_sex',
'insured_education_level',
'insured_occupation',
'insured_hobbies',
'insured_relationship',
'incident_date',
'incident_type',
'collision_type',
'incident_severity',
'authorities_contacted',
'incident_state',
'incident_city',
#'incident_location',
'property_damage',
'police_report_available',
'auto_make',
'auto_model',
'fraud_reported']


numerical_columns= ['injury_claim', 'umbrella_limit', 'insured_zip', 'property_claim', 'witnesses', 'age', 'capital-gains', 'months_as_customer', 'incident_hour_of_the_day', 'policy_number', 'policy_annual_premium', 'auto_year', 'total_claim_amount', 'number_of_vehicles_involved', 'capital-loss', 'bodily_injuries', 'vehicle_claim', 'policy_deductable']

# Step 1: StringIndexer and OneHotEncoder for categorical columns
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid="keep") for col in categorical_columns]
encoders = [OneHotEncoder(inputCol=f"{col}_index", outputCol=f"{col}_encoded") for col in categorical_columns]

# Step 2: VectorAssembler for numerical columns
num_assemblers = [VectorAssembler(inputCols=[col], outputCol=f"{col}_vec") for col in numerical_columns]

# Step 3: MinMaxScaler for numerical columns (after they are assembled into vectors)
scalers = [MinMaxScaler(inputCol=f"{col}_vec", outputCol=f"{col}_normalized") for col in numerical_columns]

# Step 5: Create the full pipeline
pipeline = Pipeline(stages=indexers + encoders + num_assemblers + scalers)



# 0311
# # Step 1: StringIndexer for categorical columns
# sex_indexer = StringIndexer(inputCol="sex", outputCol="sex_index")
# smoker_indexer = StringIndexer(inputCol="smoker", outputCol="smoker_index")
# region_indexer = StringIndexer(inputCol="region", outputCol="region_index")

# # Step 2: OneHotEncoder for region
# region_encoder = OneHotEncoder(inputCol="region_index", outputCol="region_encoded")

# # Step 3: VectorAssembler to combine "bmi" for MinMaxScaler
# bmi_assembler = VectorAssembler(inputCols=["bmi"], outputCol="bmi_assembled")
# age_assembler = VectorAssembler(inputCols=["age"], outputCol="age_assembled")

# # Step 4: MinMaxScaler for normalization
# bmi_scaler = MinMaxScaler(inputCol="bmi_assembled", outputCol="bmi_normalized")
# age_scaler = MinMaxScaler(inputCol="age_assembled", outputCol="age_normalized")

# # Step 5: Create Pipeline
# pipeline = Pipeline(stages=[
#     sex_indexer, smoker_indexer, region_indexer, region_encoder,  # Categorical encoding
#     bmi_assembler, bmi_scaler,  # bmi normalization
#     age_assembler, age_scaler   # age normalization
# ])

In [0]:
from pyspark.ml.feature import VectorAssembler

def vectorize_features(df_transformed):
    # Step 1: Select the features to vectorize
    input_columns = ["sex_index", "smoker_index", "region_encoded", "bmi_normalized", "age_normalized", "children"]

    # Step 2: Create a new VectorAssembler to combine features
    assembler = VectorAssembler(inputCols=input_columns, outputCol="features")

    # Step 3: Rename expenses to label
    df_transformed = df_transformed.withColumnRenamed("expenses", "label")

    # Step 4: Vectorize the features using VectorAssembler
    df_vector = assembler.transform(df_transformed)
    df_final = df_vector.select("label", "features")

    return df_final


In [0]:
def plot_learning_curve(X_train, y_train,rf, para, start, end, step,cv=3):
    results = []
    for i in range(start, end, step):
        rf.set_params(**{para: i})
        scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc')
        results.append(scores.mean())
    plt.plot(range(start, end, step), results, color="red", label="auc")
    plt.legend()
    plt.title(f"{para}")
    plt.show()

In [0]:
def show_shap_summary(rf, X_train, X_test, feature_names):
    explainer = shap.Explainer(rf, X_train)
    shap_values = explainer(X_test, check_additivity=False)
    shap.summary_plot(shap_values, feature_names=feature_names)

In [0]:
def show_shap_sample(X_train, X_test,feature_names, sample_idx):
    explainer = shap.Explainer(rf, X_train)
    shap_values = explainer(X_test, check_additivity=False)
    shap_values_for_sample = shap_values[sample_idx]
    return shap.force_plot(explainer.expected_value, shap_values_for_sample.values, X_test[sample_idx], feature_names=feature_names)

In [0]:
def application_predict_expenses(path_saved, age, sex, bmi, children, smoker, region, expenses):
    loaded_model = PipelineModel.load(path_saved)
    
    new_data = spark.createDataFrame([(age, sex, bmi, children, smoker, region, expenses)], ["age", "sex", "bmi", "children", "smoker", "region", "expenses"])
    
    predictions = loaded_model.transform(new_data)
    
    expected_expenses = predictions.withColumn("Expected_Expenses", round("prediction", 2))
    expected_expenses = expected_expenses.withColumn("Actual_Expenses", round("expenses", 2))

    expected_expenses = expected_expenses.withColumn("Claim_Status", when(col("Expected_Expenses") < col("Actual_Expenses"), "Abnormal Claim").otherwise("Normal Claim"))
    
    display(expected_expenses.select("Claim_Status", "Expected_Expenses", "Actual_Expenses"))

In [0]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("example").getOrCreate()


# def display(df):
#     """
#     Mimics Databricks' display() function by converting a Spark DataFrame to Pandas 
#     and displaying it nicely in Jupyter notebooks.
    
#     Args:
#         df (pyspark.sql.DataFrame): The Spark DataFrame to display.
#     """
#     try:
#         from IPython.display import display as ipy_display
#         ipy_display(df.toPandas())  # Convert to Pandas and display
#     except:
#         print(df.show())  # Fallback to show() if Pandas conversion fails